<a href="https://colab.research.google.com/github/Selii341/COSC2793A2/blob/main/A2_Sisi_Zhang_S4000130.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Data Prepeartaion

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## 1.1 Loading Data

In [ ]:
#Unzip the File
!unzip -q -o UCI-electricity.zip
!rm UCI-electricity.zip
!ls

A1                           UCI-electricity
A2_Sisi Zhang_S4000130.ipynb __MACOSX


In [ ]:
data = pd.read_csv("UCI-electricity/UCI_data.csv",delimiter=',')

In [ ]:
data.head()

,date,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,TARGET_energy
0,2016-04-19 20:30:00,22.200000,39.500000,20.566667,37.656667,22.230000,37.030000,22.318571,36.610000,20.633333,...,33.90,9.70,766.100000,65.5,3.500000,40.000000,3.350000,24.061869,24.061869,60
1,2016-03-05 04:40:00,20.356667,37.126667,17.566667,40.230000,20.890000,37.663333,18.700000,36.260000,18.463333,...,41.09,0.30,740.333333,99.0,1.000000,41.333333,0.100000,4.622052,4.622052,50
2,2016-03-14 12:40:00,20.926667,38.790000,21.100000,35.526667,21.600000,36.290000,21.000000,34.826667,18.100000,...,38.76,4.40,768.466667,72.0,6.000000,22.666667,-0.266667,5.635898,5.635898,80
3,2016-01-22 15:30:00,18.290000,38.900000,17.290000,39.260000,18.390000,39.326667,16.100000,38.790000,16.100000,...,39.20,3.35,760.600000,82.0,5.500000,41.000000,0.500000,49.216445,49.216445,40
4,2016-02-10 00:40:00,22.290000,42.333333,21.600000,40.433333,22.666667,43.363333,19.100000,40.900000,19.290000,...,43.73,3.20,738.900000,88.0,7.333333,56.000000,1.400000,47.617579,47.617579,60


The ID columns is useless here,We will drop it later.

In [ ]:
data.describe()

Describing the dataset. The dataset consists of examples from 193 different countries from different years. It is shown that minimum value for year is 2002 and maximum is 2017; it consists of 15 year period. Also the status column is binary. The population column has huge values in it, which is expected. The values can be showed as millions.However, The SLS dsitribution will be very skewed and some minumum value is zero,we will check them later.

In [ ]:
data.info()

No missing values and all datatype is constructed well.All float are the countinous number,some of int is the Category number i.e. Country,Year and Status.

In [ ]:
data.shape

Now checking shape of dataset which will return the value of number of Rows and Columns.There are 24 Columns and 2071 rows.

## 1.2 Cleaning  Data

In [ ]:
#Status checking
print(set((data['Status']).astype(int)))

In [ ]:
#Missing value
data.isnull().sum()

In [ ]:
#Population Zoom Out
data['Population'] = data['Population']/1000000
data

In [ ]:
#Rename the columns thinness  1-19 years to match the feature description
data.rename(columns={'Thinness1-19years':'Thinness10-19years'}, inplace=True)

In [ ]:
#Checking the data again
data.describe()

In total, AdultMortality, AdultMortality-Male and
AdultMortality-Female are very similar in each value.

In [ ]:
#Looking at the name of columns which can be used later.
data.columns

In [ ]:
#Checking any repeat rows which are meaningless.
data.duplicated().any()

# 2. EDA Data distribution


In [ ]:
#We will first drop the ID column as it cannot be used in our Regression model.
data = data.drop(['ID'], axis=1)
data.columns

## 2.1 Overall Distributions

In [ ]:
#Histograms
plt.figure(figsize=(30,40))
for i, col in enumerate(data.columns):
    plt.subplot(6,4,i+1)
    plt.hist(data[col], alpha=0.3, color='b', density=True)
    plt.title(col)
    plt.xticks(rotation='vertical')

These are the different histograms for all 20+ quantitative features. Some of these features will be looked into deeper.
Attribute country/status are categorical variables.
Many attributes are heavily skewed. e.g. SLS, %Expend, Measles, Under5LS, HIV-AISD...
Attributes Poilo,Diphtheria,thiness10-19/5-9... has values that are far from the majority values.
Target variable is lightly skewed and some dstirbution may used log transformation i.e.polio,dihtheria,Income and schooling.Further investigations are needed.


In [ ]:
#Distance plot for Life Expectancy column
sns.distplot(data['TARGET_LifeExpectancy'])

#Find the skewness of life expectancy
print(data['TARGET_LifeExpectancy'].skew())

Life expectancy is slightly left-skewed. This variable will not be transformed.

In [ ]:
#Colums to visualize the skews and outliers except the status which will be checked later.
columns = {1: 'TARGET_LifeExpectancy', 2: 'Country', 3: 'Year', 4: 'AdultMortality',
        5: 'AdultMortality-Male' , 6: 'AdultMortality-Female', 7: 'SLS',
       8: 'Alcohol', 9: 'PercentageExpenditure', 10: 'Measles', 11: 'BMI', 12: 'Under5LS',
       13: 'Polio', 14: 'TotalExpenditure', 15: 'Diphtheria', 16: 'HIV-AIDS',
       17: 'GDP', 18: 'Population',19: 'Thinness10-19years', 20: 'Thinness5-9years',
           21: 'IncomeCompositionOfResources', 22: 'Schooling'}

#Looking their skews
for i, column in columns.items():
                     print(i,columns[i],data[column].skew())


|skew|>0.5will be considered to transform

In [ ]:
#Box Plots to visualize the outliers
plt.figure(figsize=(28, 30))

for i, column in columns.items():
                     plt.subplot(4,6,i)
                     sns.boxplot(data[column], orient='v')
                     plt.title(column)

plt.show()


Now it is reasonable that those highly skewed attribute have lots outliers.The ouliers need be removed.

## 2.2 Categorical Features

We will use frequency tables to look at the number of times a value was repeated in each of the categorical feature. There are 2 different categorical features, so each one is explored below.

In [ ]:
#Checking the numbers in status class.
pd.set_option("display.max_rows", None)
data["Status"].value_counts()

In [ ]:
# see the impact of "status" on "Life Expectancy"

# set figure size
plt.figure(figsize=(12,6))
sns.boxplot(x = data['Status'], y = data['TARGET_LifeExpectancy'], data = data)
plt.xlabel("Status")
plt.ylabel("TARGET_LifeExpectancy")
plt.show()


This boxplot shows that country status is 1(developed) countries are stable because the range of the live expectancy age of developed countries is short, also the status 0 (developing) countries may have very different life expenctacy rates.

In [ ]:
pd.set_option("display.max_rows", None)
data["Country"].value_counts(ascending=True)

## 2.3 Ploting Heat Map to show the relation between data values

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8))
sns.heatmap(ax=ax,annot=True, fmt = ".2f",data=data.corr())

Correlation map of the features. 1 means positively correlated, -1 means negatively correlated. Income and schooling are highly positively correlated which is bigger than 0.6. Also the three adult mortalities are highly negatively correlated. I would like to keep one of the features.

## 2.4  Ploting Pair plot to see both distribution of single variables and relationships between two variables

In [ ]:
#Pair Plot
g = sns.PairGrid(data, vars=['Country', 'Year', 'AdultMortality',
       'SLS', 'Alcohol','PercentageExpenditure', 'Measles', 'BMI', 'Under5LS', 'Polio',
       'TotalExpenditure', 'Diphtheria', 'HIV-AIDS', 'GDP', 'Population',
       'Thinness10-19years', 'Thinness5-9years',
       'IncomeCompositionOfResources', 'Schooling','TARGET_LifeExpectancy'], hue="Status")
g.map(sns.scatterplot)
plt.show()

In this plot, the developed county is highly influenced at the GDP, Income, Schooling, Lifespan and Expenditures. But fewer impacts on mortalities, Thinness and other disease, this is reasonable.
On the other hands, some attributes are linear (ex. target vs schooling and adult), some are performed as non-linear (ex.target vs country and year) and most of the plots are not clear to performed as linear relationship may be considered in polynomial distribution.

In [ ]:
#Just for decalring the representation of orange and blue.
count_vals = list(data.columns)[2:]
for f in count_vals:
    sns.lmplot(data=data, x=f, y="TARGET_LifeExpectancy", hue="Status", markers=["o", "x"])
    nombre = f.translate({ord(c): None for c in string.whitespace})

In [ ]:
data.groupby('Year').mean()['TARGET_LifeExpectancy'].plot()

Through the graph, we found that life expectancy was at its lowest in 2002 and began to increase annually thereafter, with a particularly rapid increase starting in 2009. By consulting information, we learned that many natural disasters occurred worldwide in 2001-2002, and conflicts in some countries may have led to a decline in human life expectancy. After 2009, with the development of the economy and medicine, human life expectancy began to increase regularly.

### Other Data Analysis

In [ ]:
#I choosing the high correralted attribute
plt.figure(figsize=(12,6))
plt.subplot(2,3,1)
plt.scatter(x = data['Schooling'], y = data['GDP'])
plt.title("Schooling Vs GDP")

plt.subplot(2,3,2)
plt.scatter(x = data['Schooling'], y = data['IncomeCompositionOfResources'])
plt.title("Schooling Vs IncomeCompositionOfResources")

plt.subplot(2,3,3)
plt.scatter(x = data['Schooling'], y = data['HIV-AIDS'])
plt.title("Schooling Vs HIV-AIDS")

plt.subplot(2,3,4)
plt.scatter(x = data['Schooling'], y = data['Thinness10-19years'])
plt.title("Schooling Vs Thinness10-19years")

plt.subplot(2,3,5)
plt.scatter(x = data['Schooling'], y = data['Thinness5-9years'])
plt.title("Schooling Vs Thinness5-9years")

plt.subplot(2,3,6)
plt.scatter(x = data['Schooling'], y = data['AdultMortality'])
plt.title("Schooling Vs AdultMortality")


From the graph, we can observe a positive correlation between education and life expectancy. Countries with better education tend to have higher life expectancies. Additionally, there is a negative correlation between education and adult mortality rates, as well as thinness among age groups 5-19. This indicates that countries with lower levels of education tend to have higher adult mortality rates and a greater number of individuals experiencing weakness within the specified age groups.

# 3. Feature Transformation
From EDA, the following features will be transformed.

4 AdultMortality 1.0556897487743073

(AdultMortality-Male 1.0857582650450177,AdultMortality-Female 1.0456903472563532)

7 SLS 8.918731450442381

8 Alcohol 0.5931174974663874  The skewness is good but becuase of the right skewness illustrated by the histrogram, we will transform this variable.

9 PercentageExpenditure 4.783377334278013

10 Measles 11.393260539407844

12 Under5LS 8.6091945414921

13 Polio -2.0694294983318042

15 Diphtheria -2.0903544010792072

16 HIV-AIDS 5.917949827757557

17 GDP 3.582881011151258

18 Population 15.312554068769145

19 Thinness10-19years 1.7989918792788289

20 Thinness5-9years 1.8625650916241798

21 IncomeCompositionOfResources -0.9939962573022197

22 Schooling -2.265334672771308

Outlier must need to be Removed, because these values are not going to contribute while building model even they are going to despotize our Model. We can transform the data using IQR method and logarithmic scale. The advantage is that we can "normalize" the data and eliminate outliers. We cannot apply it for variables with a value less than or equal to 0.


## 3.1  Outliers removal

In [ ]:
# Transform features using the IQR method for outlier removal
from tabulate import tabulate
#IQR method for removal
def determine_outlier_thresholds_iqr(dataframe, col_name, th1=0.25, th3=0.75):
    """
    Determine the lower and upper outlier thresholds using the Interquartile Range (IQR) method.

    Args:
    - dataframe: DataFrame containing the data
    - col_name: Name of the column for which thresholds are calculated
    - th1: Lower quartile (default is 0.25)
    - th3: Upper quartile (default is 0.75)

    Returns:
    - lower_limit: Lower outlier threshold
    - upper_limit: Upper outlier threshold
    """
    quartile1 = dataframe[col_name].quantile(th1)
    quartile3 = dataframe[col_name].quantile(th3)
    iqr = quartile3 - quartile1
    upper_limit = quartile3 + 1.5 * iqr
    lower_limit = quartile1 - 1.5 * iqr
    return lower_limit, upper_limit
#Determine the Outliers
def check_outliers_iqr(dataframe, col_name):
    """
    Check for outliers using the Interquartile Range (IQR) method.

    Args:
    - dataframe: DataFrame containing the data
    - col_name: Name of the column for which outliers are checked

    Returns:
    - True if outliers are present, False otherwise
    """
    lower_limit, upper_limit = determine_outlier_thresholds_iqr(dataframe, col_name)
    if dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)].any(axis=None):
        return True
    else:
        return False
#If outlier then replace to the normalised values.
def replace_with_thresholds_iqr(dataframe,cols, th1=0.05, th3=0.95, replace=False):
    """
    Replace outliers with the threshold values using the Interquartile Range (IQR) method.

    Args:
    - dataframe: DataFrame containing the data
    - cols: List of columns for which outliers are checked and replaced
    - th1: Lower quartile (default is 0.05)
    - th3: Upper quartile (default is 0.95)
    - replace: Boolean indicating whether to replace outliers or not (default is False)

    Returns:
    - None
    """
    for col_name in cols:
        if col_name != 'Schooling':
            outliers_ = check_outliers_iqr(dataframe,col_name)
            count = None
            lower_limit, upper_limit = determine_outlier_thresholds_iqr(dataframe, col_name, th1, th3)
            if outliers_:
                count = dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)][col].count()
                if replace:
                    if lower_limit < 0:
                        # We donnot want to replace with negative values.
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
                    else:
                        dataframe.loc[(dataframe[col_name] < lower_limit), col_name] = lower_limit
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
            outliers_status = check_outliers_iqr(data, col_name)
            data.append([outliers_, outliers_status, count, col_name, lower_limit, upper_limit ])
            table = tabulate(data, headers=['Outliers (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt='rst', numalign='right')
            print("Removing Outliers using IQR")
            print(table)


In [ ]:
#Using data and columns
replace_with_thresholds_iqr(data, data.columns)

In [ ]:
# Check the updated distribution after outlier removal
plt.figure(figsize=(30,40))
for i, col in enumerate(count_vals, start=1):
    plt.subplot(6, 4, i)
    data.boxplot(col)


As there is still some Outliers left, we can ignore them as they aren't going to highly effect our Model.
We observe that Measles and Population have quite a few outliers. Of these, we suspect that they will not be very useful as estimators: measles is not a lethal disease in the vast majority of cases (nor does it harm victims much compared to other diseases deadliest), and the population we suspect will not affect quality of life much (we will later study how correlated it is with this objective metric).

In [ ]:
# Check the updated distribution after outlier removal
plt.figure(figsize=(30, 40))
for i, col in enumerate(data.columns):
    plt.subplot(6, 4, i + 1)
    plt.hist(data[col], alpha=0.3, color='b', density=True)
    plt.title(col)
    plt.xticks(rotation='vertical')

In [ ]:
#Apply logit transformation for the logit histogram
data['Polio']=np.log(((data['Polio'])/(100-data['Polio'])))
plt.figure(figsize=(12,5))
sns.displot(data['Polio'])
plt.show()
data['Diphtheria']=np.log(((data['Diphtheria'])/(100-data['Diphtheria'])))
plt.figure(figsize=(12,5))
sns.displot(data['Diphtheria'])
plt.show()

print(data['Polio'].skew(),data['Diphtheria'].skew())

The logit transformation has a better skewness we will keep that feature.

## 3.2 Additional Transform(Intern.)
Here I using the Box Cox power and use it to compare to the logit tranform(only for skew > -1) as a advanced transformation. However, this is not be used at the end based on the instructions.

In [ ]:
#function to return the Box Cox power
import scipy.stats as stat


def bcpwr (y):
    skew1 = pd.DataFrame.skew (pd.DataFrame (y))
    print ("original skewness:", skew1)
    ymin = np.min (y)
    if ymin <= 0:
        offset = np.min (y [y>0]) / 2
    else:
        offset = 0
    bc = stat.boxcox (y + offset)
    return (bc [1])
#function to plot a histogram of the data, calculate the box cox power, and plot another histogram of the power-transformed feature

def plotbc (y, plots=True):

    if plots:
        sns.histplot (y).set (title="Original Data")

    pwr = bcpwr (y)
    print ("power=", pwr)
    ymin = np.min (y)
    if ymin <= 0:
        offset = np.min (y [y>0]) / 2
    else:
        offset = 0

    if pwr < -0.75:
        bcy = -1 * (y + offset) ** -1
    elif pwr < -0.25:
        bcy = -0.5 * (y + offset) ** -0.5
    elif pwr < 0.25:
        bcy = np.log (y + offset)
    elif pwr < 0.75:
        bcy = pwr * y ** 0.5
    elif pwr < 1.25:
        bcy = y
    else:
        bcy = y ** 2

    if plots:
        plt.show()
        sns.histplot (bcy).set (title="Transformed Data")
        plt.show()

    return (bcy)

In [ ]:
#transform Alcohol
data['transformed_alcohol']= plotbc (data['Alcohol'],plots=True)
print(data['transformed_alcohol'].skew())

Alcohol now has a skewness of -0.138. This is better.

In [ ]:
#transform Alcohol
data['transformed_GDP']= plotbc (data.GDP, plots=True)

print(data['transformed_GDP'].skew())

In [ ]:
data['transformed_population']= plotbc (data['Population'], plots=True)
print(data['transformed_population'].skew())

In [ ]:
data['transformed_Thinness10-19years']= plotbc (data['Thinness10-19years'], plots=True)
print(data['transformed_Thinness10-19years'].skew())

In [ ]:
data['transformed_Thinness5-9years']= plotbc (data['Thinness5-9years'], plots=True)
print(data['transformed_Thinness5-9years'].skew())

In [ ]:
data['transformed_AdultMortality']= plotbc (data['AdultMortality'], plots=True)
print(data['transformed_AdultMortality'].skew())

In [ ]:
data['transformed_AdultMortality-Male']= plotbc (data['AdultMortality-Male'], plots=True)
print(data['transformed_AdultMortality-Male'].skew())
data['transformed_AdultMortality-Female']= plotbc (data['AdultMortality-Female'], plots=True)
print(data['transformed_AdultMortality-Female'].skew())

In [ ]:
#logit transformation of Hepatitis B
data['logit_polio']=np.log(((data['Polio'])/(100-data['Polio'])))
plt.figure(figsize=(12,5))
sns.displot(data['logit_polio'])
plt.show()
print(data['logit_polio'].skew())

Logit Polio has a skewess of -0.981.

In [ ]:
data['transformed_polio']= plotbc (data['Polio'], plots=True)
print(data['transformed_polio'].skew())

The logit transformation has a better skewness we will keep that feature and drop the boxcox transformation.

In [ ]:
data['logit_diphtheria']=np.log(((data['Diphtheria'])/(100-data['Diphtheria'])))
plt.figure(figsize=(12,5))
sns.displot(data['logit_diphtheria'])
plt.show()

data['transformed_diptheria']= plotbc (data['Diphtheria'], plots=True)
print(data['logit_diphtheria'].skew())
print(data['transformed_diptheria'].skew())

The logit transformation has a better skewness we will keep that feature and drop the boxcox transformation.

In [ ]:
data['logit_income']=np.log(((data['IncomeCompositionOfResources']+1)/(100-(data['IncomeCompositionOfResources']+2))))
plt.figure(figsize=(12,5))
sns.displot(data['logit_income'])
plt.show()

data['transformed_income']= plotbc (data['IncomeCompositionOfResources'], plots=True)
print(data['logit_income'].skew())
print(data['transformed_income'].skew())


The boxcox transformation of income has a better skewness and distribution we will keep this transformation and drop logit of income.

In [ ]:
data['logit_Schooling']=np.log(((data['Schooling']+1)/(100-(data['Schooling']+2))))
plt.figure(figsize=(12,5))
sns.displot(data['logit_Schooling'])
plt.show()

data['transformed_Schooling']= plotbc (data['Schooling'], plots=True)

print(data['logit_Schooling'].skew())
print(data['transformed_Schooling'].skew())


The boxcox transformation of schooling has a better skewness and distribution we will keep this transformation and drop logit of schooling.

In [ ]:
data['transformed_Under5LS']= plotbc (data['Under5LS'], plots=True)
print(data['transformed_Under5LS'].skew())


In [ ]:
data['transformed_HIV-AIDS']= plotbc (data['HIV-AIDS'], plots=True)
print(data['transformed_HIV-AIDS'].skew())


In [ ]:
data['transformed_PercentageExpenditure']= plotbc (data['PercentageExpenditure'], plots=True)
print(data['transformed_PercentageExpenditure'].skew())


In [ ]:
data['transformed_Measles']= plotbc (data['Measles'], plots=True)
print(data['transformed_Measles'].skew())


In [ ]:
data['transformed_SLS']= plotbc (data['SLS'], plots=True)
print(data['transformed_SLS'].skew())

In [ ]:
#drop initial and some transformed values (those used for comparison)
#data.drop('Alcohol', axis = 1, inplace=True)
#data.drop('GDP', axis = 1, inplace=True)
#data.drop('Population', axis = 1, inplace=True)
#data.drop('Thinness10-19years', axis = 1, inplace=True)
#data.drop('Thinness5-9years', axis = 1, inplace=True)
#data.drop('AdultMortality', axis = 1, inplace=True)
#data.drop('AdultMortality-Male', axis = 1, inplace=True)
#data.drop('AdultMortality-Female', axis = 1, inplace=True)
#data.drop('Polio', axis = 1, inplace=True)
#data.drop('transformed_polio', axis = 1, inplace=True)
#data.drop('Diphtheria', axis = 1, inplace=True)
#data.drop('transformed_diptheria', axis = 1, inplace=True)
#data.drop('IncomeCompositionOfResources', axis = 1, inplace=True)
#data.drop('logit_income', axis = 1, inplace=True)
#data.drop('Schooling', axis = 1, inplace=True)
#data.drop('transformed_Schooling', axis = 1, inplace=True)
#data.drop('PercentageExpenditure', axis = 1, inplace=True)
#data.drop('Measles', axis = 1, inplace=True)
#data.drop('Under5LS', axis = 1, inplace=True)
#data.drop('HIV-AIDS', axis = 1, inplace=True)
data.drop('SLS', axis = 1, inplace=True)




# 4. Model Development

## 4.1 Data Pre-process

One-hot Encoding
The following categorical features will be one-hot encoded.
Status
In general, the attribute country should be encoded but the blind test file may not have all same country data in the train dataset. This will leads to failure to predict test.

In [ ]:
#dataT = pd.get_dummies(data, columns = ['Country', 'Status',])
dataT = pd.get_dummies(data, columns = [ 'Status'])

In [ ]:
dataT.head()

In [ ]:
dataT.shape

columns number is 24 since we drop ID but split status

In [ ]:
dataT.describe()

In [ ]:
dataT.columns

In [ ]:
#Separate the target and the attributes
dataT_X = dataT.drop(['TARGET_LifeExpectancy',], axis=1).to_numpy()
dataT_y = dataT[['TARGET_LifeExpectancy']].to_numpy()


## 4.2 Data Splits

In [ ]:
#First I neet to split data to train and test

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataT_X, dataT_y, test_size=0.2, random_state=0)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=0)

#See shape of train and test data
print(f'X_train shape -->{x_train.shape}')
print(f'X_valid shape -->{x_valid.shape}')
print(f'X_test shape -->{x_test.shape}')
print(f'y_train shape -->{y_train.shape}')
print(f'y_valid shape -->{y_valid.shape}')
print(f'y_test shape -->{y_test.shape}')

### 4.2.1 Feature scales

Since we are planning to use models like linear regression which are sensitive to feature scales, standardization might be more appropriate.

In [ ]:
# Standardize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(x_train)
test_X_scaled = scaler.transform(x_test)
val_X_scaled = scaler.transform(x_valid)


In [ ]:
# Check the updated distribution after Standardize features
plt.figure(figsize=(30, 40))
for i in range(train_X_scaled.shape[1]):
    plt.subplot(6, 4, i + 1)
    plt.hist(train_X_scaled[:, i], alpha=0.3, color='b', density=True)
    plt.title(i)
    plt.xticks(rotation='vertical')

1'Country', 2'Year', 3'AdultMortality',4'AdultMortality-Male', 5'AdultMortality-Female', 6'SLS', 7'Alcohol',8'PercentageExpenditure', 9'Measles', 10'BMI', 11'Under5LS', 12'Polio',
       13'TotalExpenditure', 14'Diphtheria', 15'HIV-AIDS', 16'GDP', 17'Population',
       18'Thinness10-19years', 19'Thinness5-9years',
       20'IncomeCompositionOfResources', 21'Schooling', 22'Status_0', 23'Status_1']
       Now, most of them are standardized.

## 4.3 Model Development and Evaluation

Metrics Discussion:
MAE,MSE and RMSE are loss functions, because we want to minimize them. Here I use one of them and looking the R Square.
MAE is the easiest to understand, because it's the average error.
MSE is more popular than MAE, because MSE "punishes" larger errors, which tends to be useful in the real world.
RMSE is even more popular than MSE, because RMSE is interpretable in the "y" units.

R² is particularly useful as it represents the proportion of the variance in the dependent variable that is predictable from the independent variables.

Hence, I will using MSE and R-squared (R²).


### 4.3.1 Model 1 - Linear Regression

In [ ]:
# 1. Model Training
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()
linear_reg = linear_reg.fit(train_X_scaled, y_train)
# 2. Model Evaluation
from sklearn.metrics import mean_squared_error, r2_score
val_pred = linear_reg.predict(val_X_scaled)
val_rmse = np.sqrt (mean_squared_error(y_valid, val_pred))
val_r2 = r2_score(y_valid, val_pred)

print("Linear Regression Validation Performance:")
print(f"Validation RMSE: {val_rmse:.4f}")
print(f"Validation R^2 Score: {val_r2:.4f}")

In this case, On average, the model's predictions are off by about 4.7451 units from the actual values. And approximately 72.64% of the variance in the dependent variable (target) is predictable from the independent variables used in the model.

Let's train the data in Polynomial regression and compare their peformance.

### 4.3.2 Model 2 - Polynomial Regression

To test Polynomial Regression, I only try the degrees within 2 and 3. Beacuse it waste the running time when I defining a function in[2,3,4]  

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
# Add PolynomialFeatures
poly_features2 = PolynomialFeatures(2)
train_X_poly2 = poly_features2.fit_transform(train_X_scaled)
val_X_poly2 = poly_features2.transform(val_X_scaled)
test_X_poly2= poly_features2.transform(test_X_scaled)
# Train Polynomial Regression Model
poly_reg = LinearRegression()
poly_reg2= poly_reg.fit(train_X_poly2, y_train)
# Make predictions on the validation set
val_pred_poly2 = poly_reg2.predict(val_X_poly2)
val_rmse_poly2 = np.sqrt (mean_squared_error(y_valid, val_pred_poly2))
val_r2_poly2 = r2_score(y_valid, val_pred_poly2)
print(f"Polynomial Regression (2):")
print(f"Validation RMSE: {val_rmse_poly2:.4f}")
print(f"Validation R^2 Score: {val_r2_poly2:.4f}")

In [ ]:
# Train Polynomial Regression Model
poly_features3 = PolynomialFeatures(3)
train_X_poly3 = poly_features3.fit_transform(train_X_scaled)
val_X_poly3 = poly_features3.transform(val_X_scaled)

poly_reg = LinearRegression()
poly_reg3= poly_reg.fit(train_X_poly3, y_train)

val_pred_poly3 = poly_reg3.predict(val_X_poly3)
val_rmse_poly3 =np.sqrt ( mean_squared_error(y_valid, val_pred_poly3))
val_r2_poly3 = r2_score(y_valid, val_pred_poly3)
print(f"Polynomial Regression (3):")
print(f"Validation RMSE: {val_rmse_poly3:.4f}")
print(f"Validation R^2 Score: {val_r2_poly3:.4f}")

To comparing the evaluation we have currently got, based on the rules that A lower RMSE value indicates better model performance,and R^2 = 1 indicates a perfect fit.
Polynomial Regression (2) is chosen.

### 4.3.3 Baseline Model Plot

In [ ]:
#Looking at the coef and intercept
print("Parameter of the Polynomial model: ", poly_reg2.coef_)
print("Intercept of the Polynomial model: ", poly_reg2.intercept_)

These coefficients and intercept are essential components of the polynomial model and are used to make predictions based on the input features. They determine the shape and behavior of the polynomial curve fitted to the data during the modeling process.

In [ ]:
#Ploting the Diagnostic Plots to verify the accuray of baseline model
fig, ax = plt.subplots()
ax.scatter(y_valid, val_pred_poly2, s=25, cmap=plt.cm.coolwarm, zorder=10)

lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

ax.plot(lims, lims, 'k--', alpha=0.75, zorder=0)
ax.plot(lims, [np.mean(y_train),]*2, 'r--', alpha=0.75, zorder=0)
ax.set_aspect('equal')
ax.set_xlim(lims)
ax.set_ylim(lims)

plt.xlabel('Actual life span')
plt.ylabel('Predicted life span')

plt.show()

Generally, our model is good at predicting life expectancy. There are a few outliers though.  Ideally, most points should cluster around the ideal line, indicating accurate predictions.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_valid, y_valid-val_pred_poly2, s=25, cmap=plt.cm.coolwarm, zorder=10)

xlims = ax.get_xlim()
ax.plot(xlims, [0.0,]*2, 'k--', alpha=0.75, zorder=0)
ax.set_xlim(xlims)

plt.xlabel('Actual life span')
plt.ylabel('Residual')

plt.show()

We see a few outliers in the graph above, but apart from those there is a nice spread of data points on the predicted vs. residual plot. The large outlier beyond at around -20 is concerning.

## 4.4 Apply regularisation

Since the previous model gave us nice results for our training set but not that perfect, we can assume that the model was overfit to the train set. To try and fix this, we looked at using a Ridge Regression which is similar to a step-wise regression model and noticed at the workshop3.

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Define the Ridge regression model
ridge_reg0 = Ridge(alpha=0.1)  # adjust the alpha parameter for different regularization strength

# Train the Ridge regression model
ridge_reg0.fit(train_X_poly2, y_train)

# Make predictions on the validation set
val_pred_ridge0 = ridge_reg0.predict(val_X_poly2)

# Evaluate the model
val_rmse_ridge = np.sqrt (mean_squared_error(y_valid, val_pred_ridge0, squared=False))
val_r2_ridge = r2_score(y_valid, val_pred_ridge0)

print("Ridge Regression Validation Performance:")
print(f"Validation RMSE: {val_rmse_ridge:.4f}")
print(f"Validation R^2 Score: {val_r2_ridge:.4f}")


The Ridge regression model seems to perform well on the validation set, with lower RMSE and a decent R^2 score compared to the previous model. This indicates that regularization has helped in mitigating overfitting, resulting in a model that generalizes better to unseen data.

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Define the Ridge regression model
ridge_reg0 = Ridge(alpha=0.1)

# Train the Ridge regression model
ridge_reg0.fit(train_X_poly2, y_train)

# Make predictions on the test set
test_pred_ridge0 = ridge_reg0.predict(test_X_poly2)

# Evaluate the model
test_rmse_ridge = np.sqrt (mean_squared_error(y_test, test_pred_ridge0, squared=False))
test_r2_ridge = r2_score(y_test, test_pred_ridge0)

print("Ridge Regression Test Performance:")
print(f"Test RMSE: {test_rmse_ridge:.4f}")
print(f"Test R^2 Score: {test_r2_ridge:.4f}")


It looks like the Ridge regression model is performing well both on the validation set and the test set:

The validation RMSE (Root Mean Squared Error) is 3.4544, which indicates the average deviation of the predicted life expectancy values from the actual values.
The validation R^2 score is 0.8550, which indicates the proportion of the variance in the target variable (life expectancy) that is predictable from the features.
These results suggest that the Ridge regression model is providing a good fit to the validation data and is able to explain a significant portion of the variance in the target variable.


In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_valid,y_valid - val_pred_ridge0 , s=25, cmap=plt.cm.coolwarm, zorder=10)

xlims = ax.get_xlim()
ax.plot(xlims, [0.0,]*2, 'k--', alpha=0.75, zorder=0)
ax.set_xlim(xlims)

plt.xlabel('Actual life span')
plt.ylabel('Residual')

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test,y_test - test_pred_ridge0 , s=25, cmap=plt.cm.coolwarm, zorder=10)

xlims = ax.get_xlim()
ax.plot(xlims, [0.0,]*2, 'k--', alpha=0.75, zorder=0)
ax.set_xlim(xlims)

plt.xlabel('Actual life span')
plt.ylabel('Residual')

plt.show()

Since the previous model gave us perfect results for our training set and very bad results for our test set,Grid Search could be useful to fixing those outliers datasets.

## 4.5 Hyper-parameter setting and tuning
### 4.5.1Grid Search for Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for GridSearchCV
param_grid = {'alpha': range(0,100)}

# Initialize Ridge Regression model
ridge_reg = Ridge()

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=ridge_reg, param_grid=param_grid, cv=5)
grid_search.fit(train_X_poly2, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Reinitialize Ridge Regression with the best parameters
best_alpha = best_params['alpha']
best_ridge_reg = Ridge(alpha=best_alpha)
best_ridge_reg.fit(train_X_poly2, y_train)


In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Define the Ridge regression model
ridge_reg100 = Ridge(best_alpha)
# Train the Ridge regression model
ridge_reg100.fit(train_X_poly2, y_train)

# Make predictions on the test set
test_pred_ridge100 = ridge_reg100.predict(test_X_poly2)

# Evaluate the model
test_rmse_ridge100 = np.sqrt (mean_squared_error(y_test, test_pred_ridge100, squared=False))
test_r2_ridge100 = r2_score(y_test, test_pred_ridge100)

print("Ridge Regression99 Test Performance:")
print(f"Test RMSE: {test_rmse_ridge100:.4f}")
print(f"Test R^2 Score: {test_r2_ridge100:.4f}")

# Make predictions on the validation set
val_pred_ridge100 = ridge_reg100.predict(val_X_poly2)

# Evaluate the model
val_rmse_ridge100 = np.sqrt (mean_squared_error(y_valid, val_pred_ridge100, squared=False))
val_r2_ridge100 = r2_score(y_valid, val_pred_ridge100)

print("Ridge Regression99 Validation Performance:")
print(f"Validation RMSE: {val_rmse_ridge100:.4f}")
print(f"Validation R^2 Score: {val_r2_ridge100:.4f}")


Now, the performance on test is better and very coles to the perfomance on validation set

### 4.5.2 Backwards Feature Selection


To considering a better performance, I am tring to select the different dataX.

In [ ]:
coefficients = linear_reg.coef_

# Print coefficients along with corresponding feature names
feature_names = dataT.drop('TARGET_LifeExpectancy', axis=1).columns
coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients.flatten()})
print(coefficients_df)



In [ ]:
# Sort coefficients by absolute value to see the most influential features
coefficients_df['AbsoluteCoefficient'] = abs(coefficients_df['Coefficient'])
sorted_coefficients = coefficients_df.sort_values(by='AbsoluteCoefficient', ascending=False)
print(sorted_coefficients)
# Select the top N variables based on absolute coefficient values
N = 10 #10 here is ensuring the enough coefficients to avoiding a poor prediction.
selected_variables = sorted_coefficients['Feature'].head(N).tolist()

print("Selected variables based on coefficient magnitude:")
print(selected_variables)

In [ ]:
#Again
dataT_X2 = dataT[selected_variables].to_numpy()

#Split data with the same ratio 8:1:1
x2_train, x2_test, y2_train, y2_test = train_test_split(dataT_X2, dataT_y, test_size=0.2, random_state=0)
x2_valid, x2_test, y2_valid, y2_test = train_test_split(x2_test, y2_test, test_size=0.5, random_state=0)

# see shape of train and test data
print(f'X2_train shape -->{x2_train.shape}')
print(f'X2_valid shape -->{x2_valid.shape}')
print(f'X2_test shape -->{x2_test.shape}')
print(f'y2_train shape -->{y2_train.shape}')
print(f'y2_valid shape -->{y2_valid.shape}')
print(f'y2_test shape -->{y2_test.shape}')

In [ ]:
scaler2 = StandardScaler()
train_X2_scaled = scaler2.fit_transform(x2_train)
test_X2_scaled = scaler2.transform(x2_test)
val_X2_scaled = scaler2.transform(x2_valid)

poly2 = PolynomialFeatures(2)
train_X2_poly2 = poly2.fit_transform(train_X2_scaled)
test_X2_poly2 = poly2.transform(test_X2_scaled)
val_X2_poly2= poly2.transform(val_X2_scaled)

# Define the Ridge regression model
ridge_regX2 = Ridge(best_alpha)

# Train the Ridge regression model
ridge_regX2.fit(train_X2_poly2, y2_train)

# Make predictions on the test set
test_pred_ridgeX2 = ridge_regX2.predict(test_X2_poly2)

# Evaluate the model
test_rmse_ridgeX2 = mean_squared_error(y2_test, test_pred_ridgeX2, squared=False)
test_r2_ridgeX2 = r2_score(y2_test, test_pred_ridgeX2)

print("Ridge Regression TestX2 Performance:")
print(f"Test RMSE: {test_rmse_ridgeX2:.4f}")
print(f"Test R^2 Score: {test_r2_ridgeX2:.4f}")


 In this case, we will keep the previous ridge100 Model.

## 4.6 5-fold Cross Validation

TO verify the decision I have made, the k-fold Cross Validation is using.

In [ ]:
# 5 fold cross validation
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

# Define the MSE scorer for cross-validation
mse_scorer = make_scorer(mean_squared_error)

# Define the range of alpha parameters
alpha_values = np.logspace(-5, 5, num=10)

cv_results = dict()

# Iterate over each alpha parameter
for alpha_val in alpha_values:
    regressor = Ridge(alpha=alpha_val)

    # Perform k-fold cross-validation
    scores = cross_val_score(regressor, train_X_poly2, y_train,
                             scoring=mse_scorer, cv=5)

    cv_results[alpha_val] = scores

# Print the mean MSE for each alpha parameter
for alpha_val, scores in cv_results.items():
    print(f"Alpha Parameter: {alpha_val}")
    print(f"Mean MSE: {np.mean(scores)}")
    print("")


# Plotting mean MSE vs. alpha parameter
plt.errorbar(alpha_values, [np.mean(cv_results[alpha_val]) for alpha_val in alpha_values],
             yerr=[np.std(cv_results[alpha_val]) for alpha_val in alpha_values], fmt='-o')
plt.xscale("log")
plt.xlabel('Alpha Parameter')
plt.ylabel('Mean MSE')
plt.title('Mean MSE vs. Alpha Parameter')
plt.grid(True)
plt.show()


As the graph shown above, we are going to finding the minimum point that indicates less overfitting. The X-value in the graph is not clear but the range is from 1 to 100.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

# Define the alpha values to search
alphas = range(0,101)

# Create a Ridge regression model
ridge = Ridge()

# Perform grid search
grid_search = GridSearchCV(estimator=ridge, param_grid=dict(alpha=alphas), scoring='neg_mean_squared_error', cv=10)
grid_search.fit(train_X_poly2, y_train)  # X and y are the feature matrix and target vector

# Get the best alpha
best_alpha2 = grid_search.best_params_['alpha']
print(best_alpha2)

In [ ]:
# Define the Ridge regression model
ridge_reg75 = Ridge(best_alpha2)

# Train the Ridge regression model
ridge_reg75.fit(train_X_poly2, y_train)

# Make predictions on the validation set
test_pred_ridge75 = ridge_reg75.predict(test_X_poly2)

# Evaluate the model
test_rmse_ridge75 = np.sqrt (mean_squared_error(y_test, test_pred_ridge75, squared=False))
test_r2_ridge75 = r2_score(y_test, test_pred_ridge75)

print("Ridge Regression75 Test Performance:")
print(f"Test RMSE: {test_rmse_ridge75:.4f}")
print(f"Test R^2 Score: {test_r2_ridge75:.4f}")


# Make predictions on the validation set
val_pred_ridge75 = ridge_reg75.predict(val_X_poly2)

# Evaluate the model
val_rmse_ridge75 = np.sqrt (mean_squared_error(y_valid, val_pred_ridge75, squared=False))
val_r2_ridge75 = r2_score(y_valid, val_pred_ridge75)

print("Ridge Regression75 Validation Performance:")
print(f"Validation RMSE: {val_rmse_ridge75:.4f}")
print(f"Validation R^2 Score: {val_r2_ridge75:.4f}")


The difference between 75 and 99 is less on both sets.Based on the method I kept 99.


# 5. Predict Blind Test
After training the best Model that we declared before, the blind test data could be predicted.

## 5.1 Data Preparing
In this steps we are looking at the strcuture of the test data and repeat what we did to matching the sturcture that the model pefered.

In [ ]:
#Load data
filename2 = "test.csv"
test = pd.read_csv(filename2,delimiter=',')

In [ ]:
test.describe()

In [ ]:
#status checking
print(set((test['Status']).astype(int)))

In [ ]:
#Missing value
test.isnull().sum()

In [ ]:
test['Population'] = test['Population']/1000000
test

In [ ]:
#Rename the columns thinness  1-19 years to match the feature description
test.rename(columns={'Thinness1-19years':'Thinness10-19years'}, inplace=True)

In [ ]:
testN = test.drop(['ID'], axis=1)
testN.columns

In [ ]:
# Check the updated distribution after outlier removal
plt.figure(figsize=(30,40))
for i, col in enumerate(testN.columns):
    plt.subplot(6,4,i+1)
    plt.hist(data[col], alpha=0.3, color='b', density=True)
    plt.title(col)
    plt.xticks(rotation='vertical')

In [ ]:
#The metod is declared previously
replace_with_thresholds_iqr(testN, testN.columns)

In [ ]:
#Again using the logit transform
testN['Polio']=np.log(((testN['Polio'])/(100-testN['Polio'])))
plt.figure(figsize=(12,5))
sns.displot(test['Polio'])
plt.show()
testN['Diphtheria']=np.log(((testN['Diphtheria'])/(100-testN['Diphtheria'])))
plt.figure(figsize=(12,5))
sns.displot(test['Diphtheria'])
plt.show()

## 5.2 Apply Training Model

We will using the ridge Model within alpha 99.

In [ ]:
#We only choosing the attribute Status here.
testT= pd.get_dummies(testN, columns = [ 'Status'])
testT.head()

In [ ]:

# Standardize the features using the same scaler used during training
blind_test_scaled = scaler.transform(testT)

# Apply polynomial transformation
blind_test_poly2 = poly_features2.transform(blind_test_scaled)

# Predict life expectancy for the standardized and transformed blind test data
predictions = ridge_reg100.predict(blind_test_poly2)

# Reshape predictions array to 1-dimensional
predictions = predictions.ravel()

# Create a DataFrame with ID and predicted life expectancy
predictions_df = pd.DataFrame({'ID': test['ID'], 'LifeExpectancy': predictions})

# Save or display the predictions
print(predictions_df)


## 5.3 Prediction vertification

In [ ]:
#Distance plot for Life Expectancy column
sns.distplot(predictions_df['LifeExpectancy'])


In [ ]:
#find the skewness of life expectancy
print(predictions_df['LifeExpectancy'].skew())

To compare to the target LifeExpectancy, the predisction almost fit in the same distribution. However the range of prediction may less about 5 units than original one. Let's look at the ridge75 model.

In [ ]:

# Predict life expectancy for the standardized and transformed blind test data
predictions75 = ridge_reg75.predict(blind_test_poly2)

# Reshape predictions array to 1-dimensional
predictions75 = predictions75.ravel()

# Create a DataFrame with ID and predicted life expectancy
predictions75_df = pd.DataFrame({'ID': test['ID'], 'LifeExpectancy': predictions75})

# Save or display the predictions
sns.distplot(predictions75_df['LifeExpectancy'])
print(predictions75_df['LifeExpectancy'].skew())

In this case, although the range not really changed but the skew of 75 is less than 100 and more cloes to the train data distribution. In this case, I kept the final 75 prediction.

## 5.4 Output to a CSV file

In [ ]:
predictions75_df.to_csv('predictions.csv', index=False)